In [ ]:
import math
import time
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import random
from sklearn.metrics import confusion_matrix
import os, gc, random,cv2, warnings, math, sys, json, pprint
import seaborn as sns
import warnings
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from random import randint
import tensorflow as tf
from tensorflow.keras import models, layers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.applications import Xception
from tensorflow.keras.optimizers import Adam
from sklearn import model_selection, metrics
from tensorflow.keras import backend as K
from pylab import rcParams 
import math
from sklearn.model_selection import  GroupKFold
from sklearn.metrics import roc_auc_score


In [ ]:
dir1 = '../input/ranzcr-clip-catheter-line-classification/train'
dir2='../input/ranzcr-clip-catheter-line-classification/test'
data_dir = '../input/ranzcr-clip-catheter-line-classification'
df_train = pd.read_csv("../input/ranzcr-clip-catheter-line-classification/train.csv")
df_train_a = pd.read_csv("../input/ranzcr-clip-catheter-line-classification/train_annotations.csv")
path_dir = data_dir
sub = os.path.join(data_dir,'sample_submission.csv')
sub_df = pd.read_csv(sub)
label_cols = sub_df.columns[1:]
label_cols.values
labels = df_train[label_cols].values
train_images = dir1  + "/train/" + df_train['StudyInstanceUID'] + '.jpg'   
test_images = dir2 + "/test/" + sub_df['StudyInstanceUID'] + '.jpg'
df_train

In [ ]:
train_image=0
test_image=0
def data_image(dir1,dir2,train_image,test_image):
        dir_train = len(dir1)
        train_image += dir_train
        if dir_train:
             print(f"Train Image : {train_image}")
        dir_test  = len(dir2)
        test_image += dir_test
        if dir_test: 
             print(f"Test Image  : {test_image}")
            
data_image(dir1,dir2,train_image,test_image)

In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print(f'Running on TPU {tpu.master()}')
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy()

AUTO = tf.data.experimental.AUTOTUNE
REPLICAS = strategy.num_replicas_in_sync
print(f'REPLICAS: {REPLICAS}')

In [ ]:
def NeedleAugmentation(image, n_needles=2, dark_needles=False, p=0.5, needle_folder="../input/xray-needle-augmentation"):
    
    aug_prob = random.random()
    if aug_prob < p:
        height, width, _ = image.shape
        needle_images = [im for im in os.listdir(needle_folder) if 'png' in im]

        for _ in range(1, n_needles):
            needle = cv2.cvtColor(cv2.imread(os.path.join(needle_folder, random.choice(needle_images))), cv2.COLOR_BGR2RGB)
            needle = cv2.flip(needle, random.choice([-1, 0, 1]))
            needle = cv2.rotate(needle, random.choice([0, 1, 2]))

            h_height, h_width, _ = needle.shape
            roi_ho = random.randint(0, abs(image.shape[0] - needle.shape[0]))
            roi_wo = random.randint(0, abs(image.shape[1] - needle.shape[1]))
            roi = image[roi_ho:roi_ho + h_height, roi_wo:roi_wo + h_width]

            img2gray = cv2.cvtColor(needle, cv2.COLOR_BGR2GRAY)
            ret, mask = cv2.threshold(img2gray, 10, 255, cv2.THRESH_BINARY)
            mask_inv = cv2.bitwise_not(mask)

            img_bg = cv2.bitwise_and(roi, roi, mask=mask_inv)

            if dark_needles:
                img_bg = cv2.bitwise_and(roi, roi, mask=mask_inv)
                needle_fg = cv2.bitwise_and(img_bg, img_bg, mask=mask)
            else:
                needle_fg = cv2.bitwise_and(needle, needle, mask=mask)

            dst = cv2.add(img_bg, needle_fg, dtype=cv2.CV_64F)

            image[roi_ho:roi_ho + h_height, roi_wo:roi_wo + h_width] = dst

    return image

In [ ]:
BATCH_SIZE = 6 * REPLICAS
STEPS_PER_EPOCH = len(df_train) * 0.8 / BATCH_SIZE
VALIDATION_STEPS = len(df_train) * 0.2 / BATCH_SIZE
EPOCHS = 30
TARGET_SIZE = 750

In [ ]:
def build_decoder(with_labels = True,
                  target_size = (TARGET_SIZE, TARGET_SIZE), 
                  ext = 'jpg'):
    def decode(path):
        file_bytes = tf.io.read_file(path)
        if ext == 'png':
            img = tf.image.decode_png(file_bytes, channels = 3)
        elif ext in ['jpg', 'jpeg']:
            img = tf.image.decode_jpeg(file_bytes, channels = 3)
        else:
            raise ValueError("Image extension not supported")

        img = tf.cast(img, tf.float32) / 255.0
        img = tf.image.resize(img, target_size)

        return img
    
    def decode_with_labels(path, label):
        return decode(path), label
    
    return decode_with_labels if with_labels else decode


def build_augmenter(with_labels = True):
    def augment(img):
        img = NeedleAugmentation(img, n_needles=2, dark_needles=False, p=0.5)
        img = tf.image.random_flip_left_right(img)
        img = tf.image.random_flip_up_down(img)
        return img
    
    def augment_with_labels(img, label):
        return augment(img), label
    
    return augment_with_labels if with_labels else augment


def build_dataset(paths, labels = None, bsize = 32, cache = True,
                  decode_fn = None, augment_fn = None,
                  augment = True, repeat = True, shuffle = 1024, 
                  cache_dir = ""):
    if cache_dir != "" and cache is True:
        os.makedirs(cache_dir, exist_ok=True)
    
    if decode_fn is None:
        decode_fn = build_decoder(labels is not None)
    
    if augment_fn is None:
        augment_fn = build_augmenter(labels is not None)
    
    AUTO = tf.data.experimental.AUTOTUNE
    slices = paths if labels is None else (paths, labels)
    
    dset = tf.data.Dataset.from_tensor_slices(slices)
    dset = dset.map(decode_fn, num_parallel_calls = AUTO)
    dset = dset.cache(cache_dir) if cache else dset
    dset = dset.map(augment_fn, num_parallel_calls = AUTO) if augment else dset
    dset = dset.repeat() if repeat else dset
    dset = dset.shuffle(shuffle) if shuffle else dset
    dset = dset.batch(bsize).prefetch(AUTO)
    
    return dset

In [ ]:
(train_img, valid_img, train_labels, valid_labels) = train_test_split(train_images, labels, train_size = 0.8, random_state = 0)

In [ ]:
train_df = build_dataset(
     train_img, train_labels, bsize = BATCH_SIZE,cache = True)

valid_df = build_dataset(
     valid_img, valid_labels, bsize = BATCH_SIZE, repeat = False, shuffle = False, augment = False , cache = True)

test_df = build_dataset(test_images, bsize = BATCH_SIZE, repeat = False, shuffle = False, cache = False,augment = False)

In [ ]:
def create_model():
    conv_base = Xception(include_top = False, weights = 'imagenet',
                          input_shape = (TARGET_SIZE, TARGET_SIZE, 3))
    model = conv_base.output
    model = layers.GlobalAveragePooling2D()(model)
    model = layers.Dropout(0.26)(model)
    model = layers.Dense(11, activation = "sigmoid")(model)
    model = models.Model(conv_base.input, model)

    model.compile(optimizer = Adam(lr = 0.001),
                   loss = "binary_crossentropy",
                   metrics = [tf.keras.metrics.AUC(multi_label = True)])
    return model

In [ ]:
with strategy.scope():
    model = create_model()
    
model.summary()

In [ ]:
model_save = ModelCheckpoint('./Xcep_750_best_weights_TPU.h5', 
                             save_best_only = True, 
                             save_weights_only = True,
                             monitor = 'val_loss', 
                             mode = 'min', verbose = 1)

early_stop = EarlyStopping(monitor = 'val_loss', min_delta = 0.0001, 
                           patience = 5, mode = 'min', verbose = 1,
                           restore_best_weights = True)

reduce_lr = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.4, 
                              patience = 2, min_delta = 0.0001, 
                              mode = 'min', verbose = 1)


history = model.fit(
    train_df,
    epochs = EPOCHS,
    steps_per_epoch = STEPS_PER_EPOCH,
    validation_data = valid_df,
    validation_steps = VALIDATION_STEPS,
    callbacks = [model_save,early_stop, reduce_lr]
)

    

In [ ]:
model.save('./Xception_750_TPU.h5')

In [ ]:
model.load_weights('./Xception_750_TPU.h5')